In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly
import time
from cvxopt import matrix,solvers
from sklearn.covariance import EmpiricalCovariance, MinCovDet
import yfinance as yf
solvers.options['show_progress'] = False
solvers.options['refinement'] = 2
solvers.options['abstol'] = 1e-13
solvers.options['reltol'] = 1e-13
solvers.options['feastol'] = 1e-13

import random
import scipy
from scipy.optimize import minimize
from sklearn.neighbors import KernelDensity

Functions required for denoising

In [ ]:
def getPCA(matrix):
# Get eVal,eVec from a Hermitian matrix
    eVal,eVec=np.linalg.eigh(matrix)
    indices=eVal.argsort()[::-1] # arguments for sorting eVal desc
    eVal,eVec=eVal[indices],eVec[:,indices]
    eVal=np.diagflat(eVal)
    return eVal,eVec

def fitKDE(obs,bWidth=.25,kernel='gaussian',x=None):
    # Fit kernel to a series of obs, and derive the prob of obs
    # x is the array of values on which the fit KDE will be evaluated
    if len(obs.shape)==1:obs=obs.reshape(-1,1)
    kde=KernelDensity(kernel=kernel,bandwidth=bWidth).fit(obs)
    if x is None:x=np.unique(obs).reshape(-1,1)
    if len(x.shape)==1:x=x.reshape(-1,1)
    logProb=kde.score_samples(x) # log(density)
    pdf=pd.Series(np.exp(logProb),index=x.flatten())
    return pdf

def errPDFs(var,eVal,q,bWidth,pts=1000):
    # Fit error
    pdf0=mpPDF(var,q,pts) # theoretical pdf
    pdf1=fitKDE(eVal,bWidth,x=pdf0.index.values) # empirical pdf
    sse=np.sum((pdf1-pdf0)**2)
    return sse

def findMaxEval(eVal,q,bWidth):
    # Find max random eVal by fitting Marcenko’s dist
    out=minimize(lambda *x:errPDFs(*x),.5,args=(eVal,q,bWidth),
    bounds=((1E-5,1-1E-5),))
    if out['success']:var=out["x"][0]
    else:var=1
    eMax=var*(1+(1./q)**.5)**2
    return eMax,var

def denoisedCorr(eVal,eVec,nFacts):
    # Remove noise from corr by fixing random eigenvalues
    eVal_=np.diag(eVal).copy()
    eVal_[nFacts:]=eVal_[nFacts:].sum()/float(eVal_.shape[0]-nFacts)
    eVal_=np.diag(eVal_)
    corr1=np.dot(eVec,eVal_).dot(eVec.T)
    corr1=cov2corr(corr1)
    return corr1
    #- - - - - - - -- - - - - - - - - - - - -- - - - - - - - - - - - - -- - - - - - - - - - - - -- - -
    corr1=denoisedCorr(eVal0,eVec0,nFacts0)
    eVal1,eVec1=getPCA(corr1)
    
def atf(x):
    if isinstance(x,np.ndarray):
        x = x.item()
    return x

def mpPDF(var,q,pts=1000):
    # Marcenko-Pastur pdf
    # q=T/N
    eMin=atf(var*(1-(1/q)**.5)**2)
    eMax=atf(var*(1+(1/q)**.5)**2)

    eVal=np.linspace(eMin,eMax,pts)
    #display(eVal)
    pdf=q/(2*np.pi*var*eVal)*((eMax-eVal)*(eVal-eMin))**.5
    pdf=pd.Series(pdf,index=eVal)
    return pdf

def cov2corr(cov):
    # Derive the correlation matrix from a covariance matrix
    std=np.sqrt(np.diag(cov))
    corr=cov/np.outer(std,std)
    corr[corr<-1],corr[corr>1]=-1,1 # numerical error
    return corr


In [ ]:
class Optimiser:
    def __init__(self):
        pass
    def load_feather(self,file1,file2="",T0=0,N0=0,N=48,T=-1,normalize=False):
        self.data = pd.read_feather(file1).iloc[T0:T]
        self.universe = pd.read_feather(file2).iloc[N0:N]
        self.assets = self.data.columns[1+N0:N+1]
        # T x N matrix of closing prices 
        self.prices = np.zeros([len(self.data),len(self.assets)])            
        for i in range(len(self.assets)):
            self.prices[:,i] = self.data[self.assets[i]]
            if normalize == True:
                self.prices[:,i] = self.prices[:,i]/self.prices[0,i]
        self.price_data = self.prices
        
        # mean returns, averaged logarithmic returns
        self.mean_returns = (self.prices[-1,:] - self.prices[0,:])/len(self.prices[:,0])*1000
        self.log_returns = np.mean(np.log(self.prices[1:,:]/self.prices[:-1,:]),axis=0)
        self.naive_cov = np.cov(np.transpose(self.prices))
        #self.robust_cov = MinCovDet().fit(opt.prices).covariance_
        self.cov = self.naive_cov
        self.diffs = self.prices[1:]-self.prices[:-1]
        
    def simulate_portfolios(self, N=100, norm = 1,method="mean"):
        mean = np.zeros(len(self.assets))
        mean = opt.maximum_sharpe()[0]
        #mean = opt.minimum_variance()[0]
        lr = []
        vol = []
        r = np.random.uniform(-1,1,size=[len(self.assets),N])
        for i in range(N):
            r[:,i] = mean + r[:,i]/norm
            lr.append(Optimiser.mean_returns(self.prices,r[:,i],method))
            vol.append(Optimiser.naive_vol(self.prices,r[:,i]))
        data = {
            'returns': lr,
            'volatility': vol
        }
        return pd.DataFrame(data)
    
    def minimum_variance(self):
        var = np.cov(np.transpose(self.prices))
        a = np.ones(len(self.assets))
        x = np.linalg.solve(var,a)
        allocation = x/a.dot(x)
        return allocation, Optimiser.mean_returns(self.prices,allocation,"mean"), Optimiser.naive_vol(self.prices,allocation)
    
    def maximum_sharpe(self):
        var = np.cov(np.transpose(self.prices))
        a = self.mean_returns
        #a = self.log_returns
        x = np.linalg.solve(var,a)
        allocation = x/a.dot(x)
        #allocation = allocation/sum(allocation)
        return allocation, Optimiser.mean_returns(self.prices,allocation,"mean"), Optimiser.naive_vol(self.prices,allocation)
    
    def gradient_descent_opt(self, w0,constraints=[],cost_func = "volatility",stepmethod = ["constant",1e-6],N=100,eps = 1e-10,benchmark = 0):
        V = self.cov
        mu = self.mean_returns
        
        def cost(w):
            if cost_func == "volatility":
                return self.vol(w)
            elif cost_func == "sharpe":
                return -self.sharpe(w)
            elif cost_func == "benchmark":
                return np.linalg.norm(self.prices.dot(w)-benchmark)**2
        
        def calc_grad(w):
            if cost_func == "volatility":
                return 2*V.dot(w)
            elif cost_func == "sharpe":
                return -(mu-mu.dot(w)*V.dot(w)/self.vol(w))/np.sqrt(self.vol(w))
            elif cost_func == "benchmark":
                return 2*(self.prices.dot(w)-benchmark).dot(self.prices)
            
        def projection(x):
            if "longonly1" in constraints:
                for j in range(len(x)):
                    x[j] = abs(x[j])
            if "longonly2" in constraints:
                for j in range(len(x)):
                    x[j] = max(x[j],0)
            
            if "shortonly2" in constraints:
                for j in range(len(x)):
                    x[j] = min(x[j],0)
            
            if "sum1" in constraints:
                x = x/sum(x)
                
            if "sum-1" in constraints:
                x = -x/sum(x)
                
            return x
        
        i = 0
        w = [projection(w0)]
        grad = 10
        while i < N and np.linalg.norm(grad) > eps:
            wcurrent = w[-1]
            grad = calc_grad(wcurrent)
            p = -grad/np.linalg.norm(grad)
            
            if stepmethod[0] == "constant":
                step = np.linalg.norm(grad)*stepmethod[1]
                
            elif stepmethod[0] == "armijo":
                c = stepmethod[1]
                tau = stepmethod[2]
                m = grad.dot(p)
                step = -c*m
                if m > 0:
                    print("No descent?")
                while cost(wcurrent)-cost(wcurrent+step*p) <= -step*c*m:
                    step = tau*step
                    
            wnext = projection(wcurrent + p*step)
            w.append(wnext)
            i+=1
        return w
    
    def cardinality_opt(self,M,rho,alfa,cost_func,limit_attempts = False, attempts = 20):
        N = len(self.assets)
        T = len(self.prices)
        
        self.mean_returns = (self.prices[-1,:] - self.prices[0,:])/len(self.prices[:,0])*1000
        self.naive_cov = np.cov(np.transpose(self.prices))
        P = self.naive_cov
        q = np.matrix(-alfa*rho*self.mean_returns)
        e = np.matrix(np.ones(N))
        C = P+np.transpose(e).dot(q)+np.transpose(q).dot(e)
        return Optimiser.cardinality_optimise(C,M,limit_attempts = limit_attempts, attempts = attempts)
    
    def cardinality_benchmark(self,benchmark,M):
        N = len(self.assets)
        T = len(self.prices)
        
        P = self.prices.transpose().dot(self.prices)
        q = -np.matrix(benchmark.dot(self.prices))
        e = np.matrix(np.ones(N))
        C = P+np.transpose(e).dot(q)+np.transpose(q).dot(e)
        return Optimiser.cardinality_optimise(C,M)
        
    def l1_cardinality_benchmark(self,benchmark,cardinality,constraints):
        M = cardinality
        N = len(self.assets)
        T = len(self.prices)
        a = [1]
        done = False
        constraints.append("L1")
        ws = np.round(np.array(self.cvx_opt(constraints,"benchmark",benchmark=benchmark,parameter = a[-1])),3)
        nums = np.array([len(ws[ws>0])])
        
        i=0
        while np.all(nums > M) and i < 20:
            a.append(a[-1]*2)
            w = np.round(np.array(self.cvx_opt(constraints,"benchmark",benchmark=benchmark,parameter = a[-1])),3)
            ws = np.append(ws,w)
            nums = np.append(nums,len(w[w>0]))
            i+=1
                
        while np.all(nums < M) and i < 20:
            a.append(a[-1]/2)
            w = np.round(np.array(self.cvx_opt(constraints,"benchmark",benchmark=benchmark,parameter = a[-1])),3)
            ws = np.append(ws,w)
            nums = np.append(nums,len(w[w>0]))
            i+=1
        i = 0
        while not done:
            i+=1
            if len(a) > 1:
                if min(nums[-1],nums[-2]) <= M and M <= max(nums[-1],nums[-2]):
                    a.append((a[-1]+a[-2])/2)
                elif min(nums[-1],nums[-3]) <= M and M <= max(nums[-1],nums[-3]):
                    a[-2] = a[-3]
                    nums[-2] = nums[-3]
                    a.append((a[-1]+a[-3])/2)
                else:
                    a.append(a[-1])
            else:
                a.append(a[-1])
            w = np.round(np.array(self.cvx_opt(constraints,"benchmark",benchmark=benchmark,parameter = a[-1])),3)
            ws = np.append(ws,w)
            nums = np.append(nums,len(w[w>0]))
            
            if nums[-1] == M or i == 10:
                
                # retrieve last time n <= M to continue
                
                lastlower_index = np.where(nums<=M)[0][-1]
                a.append(a[lastlower_index])
                w = np.round(np.array(self.cvx_opt(constraints,"benchmark",benchmark=benchmark,parameter = a[-1])),3)
                ws = np.append(ws,w)
                nums = np.append(nums,len(w[w>0]))
                
                j = 0
                while nums[-1] <= M and j <= 20:
                    f = 0.99
                    w = np.round(np.array(self.cvx_opt(constraints,"benchmark",benchmark=benchmark,parameter = a[-1]*f)),3)
                    if len(w[w>0]) <= M:
                        ws = np.append(ws,w)
                        nums = np.append(nums,len(w[w>0]))
                        a.append(a[-1]*f)
                    j+=1
                done = True
        ws = ws.reshape(len(ws)//len(self.assets),len(self.assets))
        self.benchmark = benchmark
        #return ws, nums, a    
        return self.cvx_opt(cost_func="benchmark",constraints=constraints,benchmark=benchmark,sparsity=ws[-1])
        
    def monte_carlo_cardinality(self,constraints,cost_func,selection = [], benchmark = [], adjust = (0,0),debug=False, parameter = 0,minreturn = 0,maxsum=1,sparsity = [], cardinality = 0, time = [0,None], minallocation = 0,simulations=100):
        self.benchmark = benchmark
        N = len(self.assets)
        bestw = 0
        lowestcost = 1e10
        n = np.array(range(N))
        
        for j in range(simulations):
            sample = np.random.choice(n, M)
            sparsity = np.array([int(i in sample) for i in range(N)])
            w = self.cvx_opt(constraints,cost_func,selection, benchmark, adjust,debug,parameter,minreturn,maxsum,sparsity,cardinality,time,minallocation)
            c = self.cost(w,cost_func)
            lowestcost = min(lowestcost,c)
            if c == lowestcost:
                bestw = w
        return bestw
    
    def brute_cardinality(self,constraints,cost_func,selection = [], benchmark = [], adjust = (0,0),debug=False, parameter = 0,minreturn = 0,maxsum=1,sparsity = [], cardinality = 0, time = [0,None], minallocation = 0):
        N = len(self.assets)
        def power_set(input):
        # returns a list of all subsets of the list a
            if (len(input) == 0):
                return [[]]
            main_subset = [ ]
            for small_subset in power_set(input[1:]):
                main_subset += [small_subset]
                main_subset += [[input[0]] + small_subset]
            return main_subset
        sets = power_set(np.arange(0,N))
        exactsets = []
        for i in sets:
            if len(i) == cardinality:
                exactsets.append(i)
        
        curmin = 1e10
        curbestw = np.zeros(N)
        for I in exactsets:
            sparsity = np.array([int(i in I) for i in range(N)])
            w = self.cvx_opt(constraints,cost_func,selection, benchmark, adjust,debug,parameter,minreturn,maxsum,sparsity,cardinality,time,minallocation)
            cost = self.cost(w,cost_func)
            if cost <= curmin:
                curmin = cost
                curbestw = w
        return curbestw
    
    def greedy_cardinality(self,constraints,cost_func,selection = [], benchmark = [], adjust = (0,0),debug=False, parameter = 0,minreturn = 0,maxsum=1,sparsity = [], cardinality = 0, quick = True, time = [0,None],min_change = 0.0001,strict=False, minallocation = 0):
        curmin = 1e10
        curminindex = 0
        N = len(self.assets)
        n = set(np.arange(N))
        I = np.array([])
        update = True
        while I.size < cardinality and update == True:
            update = False
            if strict:
                curmin = 1e10
            for i in n:
                I_ = np.append(I,i)
                sparsity = np.array([int(i in I_) for i in range(N)])
                wi = self.cvx_opt(constraints,cost_func,selection, benchmark, adjust,debug,parameter,minreturn,maxsum,sparsity,cardinality,time,minallocation)
                costi = self.cost(wi, cost_func)
                if costi <= curmin*(1-min_change) and self.returns(wi) >= minreturn:
                    curmin = costi
                    curminindices = I_
                    addedindex = i
                    update = True
            if update:
                I = curminindices
                n.remove(addedindex)
                if debug:
                    print("Current best indices: " + str(np.sort(I)))
                    print("Current lowest cost: " + str(curmin))
            if quick == False and update:
                while update == True:
                    update = False
                    for k in I:
                        for i in n:
                            I_ = I[I!=k]
                            I_ = np.append(I_,i)
                            sparsity = np.array([int(i in I_) for i in range(N)])
                            wi = self.cvx_opt(constraints,cost_func,selection, benchmark, adjust,debug,parameter,minreturn,maxsum,sparsity,cardinality,time,minallocation)
                            costi = self.cost(wi, cost_func)
                            if costi < curmin*(1-min_change) and self.returns(wi) >= minreturn:
                                curmin = costi
                                addedindex = i
                                removedindex = k
                                curminindices = I_
                                update = True
                    if update:
                        n.remove(addedindex)
                        n.add(removedindex)
                        I = curminindices
                        if debug:
                            print("Current best indices: " + str(np.sort(I)))
                            print("Current lowest cost: " + str(curmin))
                update = True
        update = True
        
        while update == True:
            update = False
            for k in I:
                for i in n:
                    I_ = I[I!=k]
                    I_ = np.append(I_,i)
                    sparsity = np.array([int(i in I_) for i in range(N)])
                    wi = self.cvx_opt(constraints,cost_func,selection, benchmark, adjust,debug,parameter,minreturn,maxsum,sparsity,cardinality,time,minallocation)
                    costi = self.cost(wi, cost_func)
                    if costi < curmin*(1-min_change) and self.returns(wi) >= minreturn:
                        curmin = costi
                        addedindex = i
                        removedindex = k
                        curminindices = I_
                        update = True
            if update:
                n.remove(addedindex)
                n.add(removedindex)
                I = curminindices
                if debug:
                    print("Current best indices: " + str(np.sort(I)))
                    print("Current lowest cost: " + str(curmin))
        sparsity = np.array([int(i in I) for i in range(N)])
        return self.cvx_opt(constraints,cost_func,selection, benchmark, adjust,debug,parameter,minreturn,maxsum,sparsity,cardinality,time,minallocation)
            
        
    def cvx_opt(self,constraints,cost_func,selection = [], benchmark = [], adjust = (0,0),debug=False, parameter = 0,minreturn = 0,maxsum=1,sparsity = [], cardinality = 0, time = [0,None], minallocation = 0):
        #expected return, sharpe, VaR, MAD
        #cost function
        if len(benchmark) > 0:
            self.benchmark = benchmark[time[0]:time[1]]
        self.prices = self.price_data[time[0]:time[1],:]
        self.fullassets = self.assets
        self.fullprices = self.prices
        if len(sparsity) > 0:
            indices = []
            for i in range(len(self.assets)):
                if np.round(sparsity[i],5) != 0:
                    indices.append(i)
            if len(indices) == 1:
                return sparsity/max(sparsity)
            self.assets = self.fullassets[indices]
            self.prices = self.fullprices[:,indices]
            if adjust[1] != 0:
                adjust = (adjust[0][indices],adjust[1])
        
        
        N = len(self.assets)
        T = len(self.prices)
        
        
        self.mean_returns = (self.prices[-1,:] - self.prices[0,:])/len(self.prices[:,0])*1000
        self.naive_cov = np.cov(np.transpose(self.prices))
        self.cov = self.naive_cov
        self.diffs = self.prices[1:]-self.prices[:-1]
        
        if cost_func == "volatility":
            Q = np.cov(np.transpose(self.prices))
            q = np.zeros(N)
        elif cost_func == "emp_volatility":
            Q = EmpiricalCovariance().fit(self.prices).covariance_
            q = np.zeros(N)
        elif cost_func == "robust_volatility":
            Q = MinCovDet().fit(self.prices).covariance_
            q = np.zeros(N)
        elif cost_func == "denoised_volatility":
            if N >= 3:
                Q = Optimiser.denoised_cov(self.prices)
            else:
                Q = np.cov(np.transpose(self.prices))
            q = np.zeros(N)
        elif cost_func == "benchmark":
            Q = self.prices.transpose().dot(self.prices)
            q = -self.benchmark.dot(self.prices)
        elif cost_func == "expected_return":
            Q = np.zeros([N,N])
            q = -self.mean_returns
        elif cost_func == "sharpe":
            Q = np.zeros([N+1,N+1])
            Q[:N,:N] = self.cov
            q = np.zeros(N+1)
        elif cost_func == "MAD":
            T = T-1
            Q = np.zeros([N+T,N+T])
            q = np.zeros(N+T)
            q[N:] = np.ones(T)
        
        #constraints
        G = np.array([])
        h = np.array([])
        A = np.array([])
        b = np.array([])
        if "longonly" in constraints:
            G = np.append(G,-np.eye(len(self.assets)))
            h = np.append(h,np.zeros(len(self.assets)))
            
        if "sum1" in constraints:
            A = np.append(A,np.ones(len(self.assets)))
            b = np.append(b,1)
        
        if "minreturn" in constraints:
            G = np.append(G,-self.mean_returns)
            h = np.append(h,-minreturn)
        
        if "return" in constraints:
            A = np.append(A,self.mean_returns)
            b = np.append(b,minreturn)
        
        if "maxsum" in constraints:
            G = np.append(G,np.ones(len(self.assets)))
            h = np.append(h,maxsum)
        
        if "select" in constraints:
            C = np.zeros([len(selection),len(self.assets)])
            for j in range(len(selection)):
                neg = 0
                if selection[j][1][0] == "!":
                    selection[j][1] = selection[j][1][1:]
                    neg = 1
                for i in range(len(self.assets)):
                    if self.universe.iloc[i][selection[j][0]]==selection[j][1]:
                        C[j,i] = 1
                if neg == 1:
                    C[j,:] = np.abs(C[j,:]-1)
            for j in range(len(selection)):
                if selection[j][2] == "=":
                    A = np.append(A,C[j,:])
                    b = np.append(b,selection[j][3])
                if selection[j][2] == "<=":
                    G = np.append(G,C[j,:])
                    h = np.append(h,selection[j][3])
                if selection[j][2] == ">=":
                    G = np.append(G,-C[j,:])
                    h = np.append(h,-selection[j][3])
        
        if minallocation > 0:
            G = np.append(G,-np.eye(len(self.assets)))
            h = np.append(h,-np.ones(len(self.assets))*minallocation)
        
        # not compatible with sharpe
        if "adjust" in constraints:
            Q = Q + np.eye(len(self.assets))*adjust[1]
            q = q + -2*adjust[1]*np.array(adjust[0])
        
        G = G.reshape(-1,len(self.assets))
        A = A.reshape(-1,len(self.assets))
        
        if cost_func == "sharpe":
            A_ = np.zeros([len(A)+1,len(A[0])+1])
            b_ = np.zeros(len(A)+1)
            A_[:len(A),:len(A[0])] = A
            A_[:len(A),len(A[0])] = -b
            A_[len(A),:len(A[0])] = self.mean_returns
            b_[len(A)] = 1
            
            G_ = np.zeros([len(G)+1,len(G[0])+1])
            h_ = np.zeros(len(G)+1)
            G_[:len(G),:len(G[0])] = G
            G_[:len(G),len(G[0])] = -h
            G_[len(G),len(G[0])] = -1
            
            A = A_
            b = b_
            G = G_
            h = h_
        
        if cost_func == "MAD":
            A_ = np.zeros([len(A),N+T])
            A_[:,:N] = A
            G_ = np.zeros([len(G)+2*T,N+T])
            h_ = np.zeros(len(G)+2*T)
            G_[:len(G),:N] = G
            h_[:len(G)] = h
            for i in range(T):
                G_[len(G)+2*i,:N] = self.diffs[i]-self.mean_returns
                G_[len(G)+2*i,N+i] = -1
                G_[len(G)+2*i+1,:N] = -self.diffs[i]+self.mean_returns
                G_[len(G)+2*i+1,N+i] = -1
            A = A_
            G = G_
            h = h_
            
        if "L1" in constraints:
            Q_ = np.zeros([len(Q)+N,len(Q)+N])
            Q_[:len(Q),:len(Q)] = Q
            q_ = np.zeros(len(q)+N)
            q_[:len(q)] = q
            q_[len(q):] = parameter*np.ones(N)
            
            A_ = np.zeros([len(A),2*N])
            A_[:,:N] = A
            G_ = np.zeros([len(G)+2*N,2*N])
            h_ = np.zeros(len(G)+2*N)
            G_[:len(G),:N] = G
            h_[:len(G)] = h
            for i in range(N):
                G_[len(G)+2*i,N+i] = -1  
                G_[len(G)+2*i,i] = 1
                G_[len(G)+2*i+1,N+i] = -1  
                G_[len(G)+2*i+1,i] = -1
            A = A_
            G = G_
            h = h_
            Q = Q_
            q = q_
            
        if "L2" in constraints:
            Q = Q + np.eye(len(Q))*parameter
        """Q = np.matrix(Q)
        q = np.matrix(q)
        G = np.matrix(G)
        h = np.matrix(h)
        A = np.matrix(A)
        b = np.matrix(b)"""
        
        
        Q = matrix(Q)
        q = matrix(q)
        G = matrix(G)
        h = matrix(h)
        A = matrix(A)
        b = matrix(b)
        
        
        if cost_func in ["MAD","expected_return"]:
            sol = solvers.lp(q,G,h,A,b)
        else:
            #for i in (Q, q, G, h, A, b):
            #    print(i)
            sol = solvers.qp(Q, q, G, h, A, b)
        
        
        
        
        #if debug == True:
        #    solution = sol
        if cost_func == "sharpe":
            solution = np.array(sol['x'])[:-1,0]/np.array(sol['x'])[-1,0]
        elif cost_func == "MAD" or "L1" in constraints:
            solution = np.array(sol['x'])[:N,0]
        else:
            solution = np.array(sol['x'])[:,0]
        
        if len(sparsity) > 0:
            sol = np.zeros(len(self.fullassets))
            for i in range(len(indices)):
                sol[indices[i]] = solution[i]
            solution = sol
        self.assets = self.fullassets
        self.prices = self.fullprices
        
        if "cardinality" in constraints:
            reduced_w = np.zeros(len(solution))
            for i in range(cardinality):
                reduced_w[np.argmax(solution)] = np.max(solution)
                solution[np.argmax(solution)] = 0
            if "sum1" in constraints:
                return reduced_w/sum(reduced_w)
            return reduced_w
        return solution
        
    def cost(self,x,cost_func,time=[0,None]):
        if cost_func == "volatility":
            Q = np.cov(np.transpose(self.prices))
            return x.dot(Q).dot(np.transpose(x))
        elif cost_func == "emp_volatility":
            Q = EmpiricalCovariance().fit(self.prices).covariance_
            return x.dot(Q).dot(np.transpose(x))
        elif cost_func == "robust_volatility":
            Q = MinCovDet().fit(self.prices).covariance_
            return x.dot(Q).dot(np.transpose(x))
        elif cost_func == "denoised_volatility":
            if len(x) >= 3:
                Q = Optimiser.denoised_cov(self.prices)
            else:
                Q = np.cov(np.transpose(self.prices))
            return x.dot(Q).dot(np.transpose(x))
        elif cost_func == "benchmark":
            return np.linalg.norm(self.prices.dot(x)-self.benchmark)
        elif cost_func == "expected_return":
            Q = np.zeros([N,N])
            q = -self.mean_returns
            x.dot(Q).dot(np.transpose(x))+2*x.dot(q)
        elif cost_func == "MAD":
            self.diffs = self.prices[1:]-self.prices[:-1]
            #display([abs(self.diffs[i]-self.mean_returns.dot(x)) for i in range(len(self.assets)-1)])
            return sum([abs(self.diffs[i].dot(x)-self.mean_returns.dot(x)) for i in range(len(self.assets)-1)])
        
    def lowerbound(C,K):
            alfa = np.min(C)
            d = sorted(np.diag(C))
            if alfa in d:
                return alfa
            K = min(K,len(d))
            return alfa + 1/np.sum(1/(d[:K]-np.ones(K)*alfa))
        
    def cardinality_optimise(C,M,limit_attempts = True, attempts = 20):
        M = M+1
        def submatrix(M,indices):
            return M[np.ix_(sorted(indices),sorted(indices))]

        def C_I(C,I):
            res = np.zeros([len(C),len(C)])
            for i in I:
                for j in I:
                    res[i,j] = C[i,j]
            return res

        def is_pd(K):
            try:
                np.linalg.cholesky(K)
                return 1 
            except np.linalg.linalg.LinAlgError as err:
                if 'Matrix is not positive definite' in err.args[0]:
                    return 0
                else:
                    raise

        # optimise x'Cx under the constraint supp(x) <= M
        n = len(C)
        N = np.arange(n)
        # index sets
        S_next = [[i] for i in N]
        mins = [np.min(np.diag(C))]
        minargs = [tuple([np.argmin(np.diag(C))])]
        minvecs = [[1]]
        
        # iterate
        for i in range(2,M):
            update = False
            S = S_next.copy()
            S_next = []
            curmin = mins[-1]
            curminargs = minargs[-1]
            curminvec = minvecs[-1]
            done = set()
            for I in S:
                J = np.setdiff1d(N,I)
                # find pos def candidates
                cand = J
                for j in J:
                    k = 0
                    while k < len(I):
                        if is_pd(submatrix(C,[j,I[k]])):
                            k+=1
                        else:
                            k = len(I)
                            cand = cand[cand!=j]
                lower = Optimiser.lowerbound(submatrix(C,np.union1d(cand,I)),M)
                if lower < curmin:
                    if limit_attempts:
                        cand = np.random.choice(cand,min(attempts,len(cand)),replace=False)
                    for j in cand:
                        # find next point
                        I_next = tuple(np.sort(np.append(I,j)))
                        if I_next not in done:
                            done.add(I_next)
                            C_I = submatrix(C,I_next)
                            e = np.ones(len(I_next))
                            xmin = (e.dot(np.linalg.inv(C_I)).dot(np.transpose(e)))**(-1)*(np.linalg.inv(C_I)).dot(np.transpose(e))
                            if np.all((xmin>0)):
                                xcost = xmin.dot(C_I).dot(np.transpose(xmin))
                                #display([I,xcost,curmin])
                                curmin = min(curmin,xcost)
                                if curmin == xcost:
                                    curminargs = I_next
                                    curminvec = xmin
                                    update = True
                                S_next.append(I_next)
                else:
                    #display("lower bound used")
                    pass

            if update:
                mins.append(np.array(curmin))
                minargs.append(curminargs)
                minvecs.append(np.array(curminvec))
            #display([curmin,curminargs,curminvec])
        
        w = np.zeros(n)
        #display("minargs: " + str(minargs) + ". minvecs:"+ str(minvecs))
        for i in range(len(minargs[-1])):
            w[minargs[-1][i]] = minvecs[-1][0][i]
        return w
   
    
    def time_subset_portfolio(self,t0,dt,constraints,cost_func,selection = [], benchmark = 0):
        self.prices_temp = self.prices
        self.prices = self.prices_temp[t0:t0+dt]
        optw = opt.cvx_opt(constraints,cost_func,selection)
        self.prices = self.prices_temp
        return optw
    
    def portfolio_sum(self,allocation,selection):
        s = 0
        for i in range(len(allocation)):
            if self.universe.iloc[i][selection[0]] == selection[1]:
                s += allocation[i]
        return s
    # volatility of allocation
    def vol(self,allocation):
        return allocation.dot(self.cov).dot(allocation)
    
    # sharpe ratio of allocation
    def sharpe(self,allocation):
        return allocation.dot(self.mean_returns)/np.sqrt(self.vol(allocation))
    
    def returns(self, allocation):
        self.mean_returns = (self.prices[-1,:] - self.prices[0,:])/len(self.prices[:,0])*1000
        return allocation.dot(self.mean_returns)
    
    def show_returns(self, allocation):
        pass
    
    def view_portfolio(self,allocation,rounding = 5, zeros = False):
        df = self.universe.filter(['TICKER','NAME','SECTOR','LOCATION'], axis=1)
        df["ALLOCATION"] = np.round(allocation,rounding)
        if zeros == False:
            df = df[df.ALLOCATION != 0]
        print("returns:" + str(opt.returns(allocation)))
        print("variance:" + str(opt.cost(allocation,"volatility")))

        return df
    
    def mad(self,data):
        m = np.mean(data)
        return np.mean(abs(m-data))
    
    def running_portfolio_returns(self,allocations):
        returns = np.zeros(len(self.prices))
        for i in range(1,len(returns)):
            returns[i] = returns[i-1] + (self.prices[i]-self.prices[i-1]).dot(allocations[i])
        return returns
    # Average return from historical data given allocation of assets
    @staticmethod
    def mean_returns(prices,allocation,method="mean"):
        if method == "mean":
            final_value = allocation.dot(prices[-1,:])
            initial_value = allocation.dot(prices[1,:])
            return (final_value-initial_value)/len(prices[:,0])
        elif method == "dailylog":
            values = prices.dot(allocation)
            return np.mean(np.log(values[1:]/values[:-1]))
        elif method == "fulllog":
            final_value = allocation.dot(prices[-1,:])
            initial_value = allocation.dot(prices[1,:])
            return np.log(final_value/initial_value)/len(prices[:,0])
    
    # Naive volatility
    @staticmethod
    def naive_vol(prices,allocation):
        var = np.cov(np.transpose(prices))
        return allocation.dot(var).dot(allocation)
    
    @staticmethod
    def denoised_cov(prices):
        T = len(prices)
        N = len(prices[0,:])
        corr0 = np.corrcoef(np.transpose(prices))
        eVal0,eVec0=getPCA(corr0)
        sigma = np.sqrt(np.diag(np.cov(np.transpose(prices))))
        q = T/N
        eMax0,var0=findMaxEval(np.diag(eVal0),q,bWidth=.01)
        nFacts0=max(eVal0.shape[0]-np.diag(eVal0)[::-1].searchsorted(eMax0),1)
        corr1 = denoisedCorr(eVal0,eVec0,nFacts0)
        return corr1*(np.outer(sigma,sigma))

    
    

## Speed/accuracy comparison of algorithms

In [ ]:
opt = Optimiser()
optbench = Optimiser()
T = 1000

opt.load_feather("eurostoxx_prices","eurostoxx_universe",T=T,N=48,normalize=True)
optbench.load_feather("SXXP_Benchmark_Prices","SXXP_Benchmark_Prices",T=T,normalize=True)

In [ ]:
### MINIMUM VARIANCE PORTFOLIO
Ms = [3,4,5,6,7,8]
a = len(Ms)
cost = []
runtime = []
for M in Ms:
    display(M)
    times = np.zeros(5)
    costs = np.zeros(5)
    
    tic = time.perf_counter()
    #wbrute = opt.brute_cardinality(cost_func="volatility",constraints=["longonly","sum1"],cardinality=M)
    toc = time.perf_counter()
    times[0] = toc-tic
    #costs[0] = opt.cost(wbrute,"volatility")
    costs[0] = 1
    
    tic = time.perf_counter()
    wrandom = opt.monte_carlo_cardinality(cost_func="volatility",constraints=["longonly","sum1"],cardinality=M,simulations=1000)
    toc = time.perf_counter()
    times[1] = toc-tic
    costs[1] = opt.cost(wrandom,"volatility")

    tic = time.perf_counter()
    wincset = opt.cardinality_opt(M,0,0,cost_func="volatility",limit_attempts=False)
    toc = time.perf_counter()
    times[2] = toc-tic
    costs[2] = opt.cost(wincset,"volatility")

    tic = time.perf_counter()
    wincsetfast = opt.cardinality_opt(M,0,0,cost_func="volatility",limit_attempts=True,attempts=5)
    toc = time.perf_counter()
    times[3] = toc-tic
    costs[3] = opt.cost(wincsetfast,"volatility")

    tic = time.perf_counter()
    wgreedy = opt.greedy_cardinality(cost_func="volatility",constraints=["longonly","sum1"],cardinality=M)
    toc = time.perf_counter()
    times[4] = toc-tic
    costs[4] = opt.cost(wgreedy,"volatility")
    cost.append(costs)
    runtime.append(times)

In [ ]:
### BENCHMARK PORTFOLIO
Ms = [5,10,15,20,25,30]
a = len(Ms)
cost = []
runtime = []
bench = np.array(optbench.prices[:,0])
for M in Ms:
    display(M)
    times = np.zeros(4)
    costs = np.zeros(4)
    
    tic = time.perf_counter()
    #wbrute = opt.brute_cardinality(cost_func="benchmark",constraints=[""],cardinality=M,benchmark=bench)
    toc = time.perf_counter()
    times[0] = toc-tic
    #costs[0] = opt.cost(wbrute,"benchmark")
    costs[0] = 10
    
    tic = time.perf_counter()
    wrandom = opt.monte_carlo_cardinality(cost_func="benchmark",constraints=[""],cardinality=M,simulations=1000,benchmark=bench)
    toc = time.perf_counter()
    times[1] = toc-tic
    costs[1] = opt.cost(wrandom,"benchmark")

    tic = time.perf_counter()
    wl1 = opt.l1_cardinality_benchmark(constraints=[""],cardinality=M,benchmark=bench)
    toc = time.perf_counter()
    times[2] = toc-tic
    costs[2] = opt.cost(wl1,"benchmark")

    tic = time.perf_counter()
    wgreedy = opt.greedy_cardinality(cost_func="benchmark",constraints=[""],cardinality=M,benchmark=bench)
    toc = time.perf_counter()
    times[3] = toc-tic
    costs[3] = opt.cost(wgreedy,"benchmark")
    cost.append(costs)
    runtime.append(times)

In [ ]:
## COMPUTE AND DISPLAY ERRORS
relcost = []
for i in cost:
    relcost.append((i-min(i))/min(i))
for i in range(len(runtime)):
    text = "&"
    for j in range(len(relcost[i])):
        text = text + str(np.round(relcost[i][j],3)) + " (" + str(np.round(runtime[i][j],1)) + "s)" + "&"
    print(text[:-1])


## Stability (allocation evolution)

In [ ]:
opt = Optimiser()
optbench = Optimiser()
T = 1000

opt.load_feather("eurostoxx_prices","eurostoxx_universe",T=T,N=48,normalize=True)
optbench.load_feather("SXXP_Benchmark_Prices","SXXP_Benchmark_Prices",T=T,normalize=True)

In [ ]:
N = 100
sample = 350
weights = np.zeros([len(opt.assets),N])
cardweights = np.zeros([len(opt.assets),N])
robustweights = np.zeros([len(opt.assets),N])
denoisedweights = np.zeros([len(opt.assets),N])

bench = optbench.prices[:,0]
M = 2
constraints = ["sum1","longonly"]

last = len(opt.prices)-sample
dt = last//N
t = np.arange(N)*dt+sample
display(t)
sparsity=np.array([0,0,1,0,0,1,0,0,0,0])
l = 0.0000

cost = "volatility"

cardweights[:,0] = opt.greedy_cardinality(debug=False,cardinality=M,time=[t[0]-sample,t[0]],cost_func=cost,constraints=constraints)
weights[:,0] = opt.cvx_opt(time=[t[0]-sample,t[0]],cost_func=cost,constraints=constraints)
robustweights[:,0] = opt.cvx_opt(time=[t[0]-sample,t[0]],cost_func="robust_volatility",constraints=constraints,minreturn=0.01)
denoisedweights[:,0] = opt.cvx_opt(time=[t[0]-sample,t[0]],cost_func="denoised_volatility",constraints=constraints,minreturn=0.01)

noadjustweights[:,0] = opt.cvx_opt(time=[t[0]-sample,t[0]],cost_func="volatility",constraints=constraints[:2])
for i in range(1,len(t)):
    if i % 5 == 0:
        print(str(i))
    cardweights[:,i] = opt.greedy_cardinality(adjust = (cardweights[:,i-1],l),debug=False,cardinality=M,time=[t[i]-sample,t[i]],cost_func=cost,constraints=constraints)
    weights[:,i] = opt.cvx_opt(adjust = (weights[:,i-1],l),time=[t[i]-sample,t[i]],cost_func=cost,constraints=constraints)
    robustweights[:,i] = opt.cvx_opt(adjust = (weights[:,i-1],l),time=[t[i]-sample,t[i]],cost_func="robust_volatility",constraints=constraints)
    denoisedweights[:,i] = opt.cvx_opt(adjust = (weights[:,i-1],l),time=[t[i]-sample,t[i]],cost_func="denoised_volatility",constraints=constraints)
    


In [ ]:
for i in range(len(opt.assets)):
    fig = px.line(labels={"x":"start time (days)","y":"portfolio weight"})
    
    fig.add_scatter(x=t,y=np.round(cardweights[i,:],5),name="cardinality")
    fig.add_scatter(x=t,y=np.round(weights[i,:],5),name="sample",marker = {'color' : 'blue'})
    fig.add_scatter(x=t,y=np.round(robustweights[i,:],5),name="robust",marker = {'color' : 'red'})
    fig.add_scatter(x=t,y=np.round(denoisedweights[i,:],5),name="denoised",marker = {'color' : 'green'})

    fig.update_layout(title="Asset " + str(i+1) + ": " + str(opt.universe.iloc[i]["NAME"]),
        font_family="Times New Roman",
        font_color="black",
        font_size=25,
        showlegend=True
        ,yaxis_range=[0,1]
        )
    fig.show()
    #opt.view_portfolio(np.round(weights[:,-1],5))
    #fig.write_image("asset" + str(i) +".png")

## L1 algorithm: assets in function of lambda

In [ ]:
opt = Optimiser()
optbench = Optimiser()
T = 1000
opt.load_feather("eurostoxx_prices","eurostoxx_universe",T=T,N=48,normalize=True)
optbench.load_feather("SXXP_Benchmark_Prices","SXXP_Benchmark_Prices",T=T,normalize=True)

In [ ]:
naive_allocation = np.ones(len(opt.assets))/len(opt.assets)
random_allocation = np.random.uniform(0,1/len(opt.assets),len(opt.assets))
bench_naive = opt.prices.dot(naive_allocation)
bench_random = opt.prices.dot(random_allocation)
bench_flat = np.ones(len(opt.prices))
bench_line = np.arange(len(opt.prices))
bench_wave = np.sin(np.arange(len(opt.prices))/300)*100

bench = optbench.prices[:,0]

params = np.logspace(-2,3,100)
nums = []
errs1 = []
errs2 = []

for i in params:
    
    minw1 = np.round(opt.cvx_opt(cost_func="benchmark",constraints=["L1","longonly"],parameter=i,benchmark = bench),3)
    w = opt.cvx_opt(cost_func="benchmark",constraints=["longonly"],benchmark=bench,sparsity=minw1)
    nums.append(len(minw1[minw1>0]))
    errs1.append(np.linalg.norm(opt.prices.dot(w)-bench))


#fig.add_scatter(x=params,y=errs2)

In [ ]:
wmin = opt.cvx_opt(cost_func="benchmark",constraints=["longonly"],benchmark=bench)
cmin = np.linalg.norm(opt.prices.dot(wmin)-bench)

In [ ]:
fig = px.line(x=params,y=nums,log_x=True)
fig.add_scatter(x=params,y=(errs1-cmin)/cmin)
fig.update_layout(title="Number of assets in optimal portfolio",
        font_family="Times New Roman",
        font_color="black",
        font_size=25,
        xaxis_title="$\lambda$",
        yaxis_title="n"
        )

## Marcenko-Pastur

In [ ]:
def getRndCov(nCols,nFacts):
    w=np.random.normal(size=(nCols,nFacts))
    cov=np.dot(w,w.T) # random cov matrix, however not full rank
    cov+=np.diag(np.random.uniform(size=nCols)) # full rank cov
    return cov


alpha,N,nFact,T=.995,1000,100,20000
cov=np.cov(np.random.normal(size=(T,N)),rowvar=0)
cov=alpha*cov+(1-alpha)*getRndCov(N,nFact) # noise+signal
corr0=cov2corr(cov)
eVal0,eVec0=getPCA(corr0)

In [ ]:
q = T/N
eMax0,var0=findMaxEval(np.diag(eVal0),q,bWidth=.01)
nFacts0=eVal0.shape[0]-np.diag(eVal0)[::-1].searchsorted(eMax0)

In [ ]:
density = mpPDF(var0,q)
fig = px.histogram(np.diag(eVal0), histnorm='probability density',nbins=1000)
fig.add_scatter(x=density.index,y=density)
fig.update_layout(title="Eigenvalue distribution (noise+signal)",
        font_family="Times New Roman",
        font_color="black",
        font_size=25,
        showlegend=False
        ,xaxis_range=[0,8]
        ,xaxis_title="$\lambda$"
        ,autosize=False
        ,width=700
        ,height=500
        )
fig.show()
#fig.write_image("mpnoise.png")